# Protein embeddings in practice

In this notebook, we will practically explore protein language models and the embeddings that they produce. We will use two freely available software packages: the comprehensive *'bio_embeddings'* package and the more recent *'protein_sequence_models'* package.

### 0. Installation and libraries
---

First, let's install the necessary software and load the desired packages. Note: the pip's dependency resolver ERROR can be safely ignored.

In [1]:
# Install software for Google Colab
!pip3 install -U pip > /dev/null
!pip3 install -U bio_embeddings[all] > /dev/null
!pip install scikit_learn==1.0.2
!pip install pyyaml==5.4.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [1]:
# Install software for Kaggle
!pip install bio_embeddings==0.2.2
!pip install torchvision==0.10.1

  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
ERROR: Invalid requirement: 'scikit_learn=1.0.2'
Hint: = is not a valid operator. Did you mean == ?


In [3]:
# import libraries
import numpy as np
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO
from bio_embeddings.embed import ProtTransBertBFDEmbedder

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


### 1. Very simple introduction
---

#### The basics

Let's use the ProtTransBertBFDEMbedder from the bio_embeddings package and compute embeddings for a simple test sequence. The basic workflow to compute an embedding is the following: (1) define an embedder (a pretrained model to compute embeddings from), (2) define your sequence as a string and (3) compute the embedding and reduce if desired. Note: The first time an embedder is loaded into memory takes a minute or two.

In [4]:
embedder = ProtTransBertBFDEmbedder()
sequence = "THISISASEQUENCE"
embedding = embedder.embed(sequence)
reduced_embedding = embedder.reduce_per_protein(embedding)

Some weights of the model checkpoint at /root/.cache/bio_embeddings/prottrans_bert_bfd/model_directory were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


> **To-do and questions**
> - Look at the embedding and reduced_embedding
> - How do these two differ? And what do they signify?
> - Recalling the theory slides, how does the reduction work? Can you think of other ways of doing this?
> - Compare the embedding of the first 'E' with the embedding of the last 'E', are they the same?

#### Computing many embeddings in a loop

If you have many sequences available for which you want to compute embeddings, it's probably easier to put them in a FASTA file and loop over them.

> **To-do**
>
> 1. Write a small function that parses a FASTA file and computes embeddings for each of the sequences.
> 
> 2. Upload the two FASTA files in the GitHub repo onto the Kaggle or Colab platform. On Kaggle, see 'Add data' button in the upper right corner. This file will be located at '../input/a_folder_you_designate'. In Colab, files can be uploaded just as easy. Click on the *folder* icon in the left bar, then select the location where you want to store your files and click on the *upload* icon.
>
> 3. Run your function on one of the uploaded files and inspect the results.
>
> Tip 1: you can use 'SeqIO.parse' and a simple list comprehension to compute your embeddings in a loop.
>
> Tip 2: to track progress, you can use the 'tqdm' function in the list comprehension.

In [ ]:
def compute_protein_embeddings(fasta_file):
  ...

In [ ]:
embeddings = compute_protein_embeddings('your_fasta_file')

#### Which is the best protein language model?


What's clear from the [bio_embeddings GitHub repo](https://github.com/sacdallago/bio_embeddings) is that a variety of different protein language models exist and can be used to compute embeddings with. However, which of these embeddings is most informative? As mentioned in the 'bio_embeddings' ReadMe: "The models prottrans_t5_xl_u50, esm1b, esm, prottrans_bert_bfd, prottrans_albert_bfd, seqvec and prottrans_xlnet_uniref100 were all trained with the goal of systematic predictions. From this pool, we believe the optimal model to be prottrans_t5_xl_u50, followed by esm1b."

Below, try to load an instance of the t5_xl_u50 model and see what you get. 

**Warning**: this takes a while... 

**Spoiler**: it crashes the CPU...

In [ ]:
from bio_embeddings.embed import ProtTransT5XLU50Embedder
embedder = ProtTransT5XLU50Embedder()

Unfortunately, notebook environments like this are not suited to run the largest, most performant models (yet), due to their size and necessary allocation on the CPU and GPU. For more info, see [the init file](https://github.com/sacdallago/bio_embeddings/blob/develop/bio_embeddings/embed/__init__.py) of the embedders.

### 2. Visualizing embeddings
---

#### Individual amino acid embeddings

At the most basic level we would assume that, in some way at least, similar amino acids have similar embeddings. A simple way to check this is to make visualizations of the embeddings in two or three dimensions. We can simply use our favorite dimensionality reduction technique (PCA, t-SNE, UMAP) for this.

> **To-do**
>
> - Compute an embedding for every naturally occuring amino acid and project them into lower dimensions.
> - Visualize these lower dimension projections. You can use the 'render_scatter_plotly' function in bio_embeddings.visualize for this. Tip: help(render_scatter_plotly) to see what is needed as input.
> - Color the amino acids by polarity and charge to identify any clusters. Tip: see the 'label' input for the render_scatter_plotly function.


In [6]:
from bio_embeddings.visualize import render_scatter_plotly
from sklearn.manifold import TSNE
import umap

In [ ]:
amino_acids = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
groups = {'polar': ['S', 'T', 'Y', 'N', 'Q'], 'non-polar':['G', 'A', 'V', 'C', 'P', 'L', 'I', 'M', 'W', 'F'], 
         'pos+': ['L', 'R', 'H'], 'neg-':['D', 'E']}
help(render_scatter_plotly)

In [ ]:
embedder = ProtTransBertBFDEmbedder()
embeddings = ...

> **To-do and Questions**
>
> - Do we see what we expect to see? What is in line with expectations, what is not?
> - Explore other embedders to see if they produce different results: BeplerEmbedder, SeqVecEmbedder, ...

#### Full protein sequence embeddings related to *Saccharomyces cerevisiae* and *Homo sapiens*

Now, we'll do the same for full proteins. More specifically, we will visualize the protein sequences in the FASTA files, collected from UniProt and related to *Saccharomyces cerevisiae* and *Homo sapiens*.

> **To-do**
> 
> - Let's get back to the two FASTA files we've seen before, upload both of them from the GitHub repo onto Kaggle or Colab if you haven't already.
> - Repeat the process above: (1) use your previously implemented function to compute embeddings for all of the protein sequences, (2) project those into lower dimensions and (3) visualize them.


In [ ]:
embeddings_sapiens = compute_protein_embeddings('../uniprot_sapiens.fasta')
embeddings_yeast = compute_protein_embeddings('../uniprot_yeast.fasta')

### 3. Stacking a supervised model
---

#### A simple binary classifier

Recalling the theory slides, we can stack a supervised machine learning model on these computed embeddings to perform a specific task. This means we use our computed embeddings as input for a supervised model where labels are available.

> **To-do**
> 
> Build a binary classifier on top of the computed embeddings that discriminate between proteins from *Homo sapiens* and *Saccharomyces cerevisiae*.
>
> Tip: you can probably build a simple classifier in Scikit-learn with [just a few lines of code](https://scikit-learn.org/stable/tutorial/basic/tutorial.html). If you're stuck, you can have a look at a [related example](https://github.com/sacdallago/bio_embeddings/blob/develop/notebooks/deeploc_machine_learning.ipynb) from the *bio_embeddings* package.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, pairwise_distances

In [ ]:
...

> **Question**
>
> How would this approach change if we were to use individual amino acid embeddings to make some predictions (e.g. predict secondary structures at the AA level)?
>

#### K-nearest neighbors

Let's try a different approach. You can also implement a simple K-nearest neigbor classifier by computing distances in the embeddings space. 

> **To-do**
> 
> 1. Split the computed embeddings into a training and test part.
> 2. Compute the pairwise distances between the training embeddings vectors and each of the test vectors. Tip: look at the above uploaded modules.
> 3. For each of the test vectors, collect the k closest training vectors and their corresponding labels.
> 4. Apply a majority vote to predict the label.
> 5. Compare the accuracy with that of your previous classifier.

In [ ]:
...

### 4. Beyond the *bio_embeddings* package
---

For those who hadn't had enough already, a few more interesting publicly available packages exist. Two that stand out are Microsoft's [*protein-sequence-models*](https://github.com/microsoft/protein-sequence-models) package and Brandes' [*protein_bert*](https://github.com/nadavbra/protein_bert) package.


> **To-do**
>
> Explore the pretrained CARP models in the *protein-sequence-models* package and how you can compute embeddings with those. Redo some of the visualizations and/or model stacking and compare.
>

In [ ]:
!pip install sequence-models
!pip install wget

In [ ]:
from sequence_models.pretrained import load_model_and_alphabet
model, collater = load_model_and_alphabet('carp_76M')

In [ ]:
...